In [1]:
# SPARK SESSIONS

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, desc
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *
from pyspark.sql.functions import col, dayofmonth, month, year, date_format
import findspark
import zipfile
import os
from pyspark.sql.functions import when, lit, col, current_timestamp, input_file_name
import pandas as pd
import requests
import tarfile
import requests
import json
from pyspark.sql import DataFrame
from pyspark.sql.session import SparkSession
from pyspark.sql.types import DoubleType
import psycopg2
# from psycopg2 import sql

/tmp/ipykernel_2831/2576859890.py:12: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# import os
# import findspark
# from pyspark.sql import SparkSession
print("SPARK_HOME:", os.environ.get('SPARK_HOME'))
print("JAVA_HOME:", os.environ.get('JAVA_HOME'))

findspark.init()
spark = SparkSession.builder.appName("sparksql").getOrCreate()


SPARK_HOME: /opt/spark
JAVA_HOME: /usr/lib/jvm/java-17-openjdk-amd64


your 131072x1 screen size is bogus. expect trouble
26/01/11 15:41:51 WARN Utils: Your hostname, r resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/01/11 15:41:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 15:41:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
try:
    spark.stop()
except:
    pass

In [ ]:
# %load_ext sparksql_magic
# %config SparkSql.max_num_rows=20

In [3]:
%load_ext sparksql_magic
%config SparkSql.max_num_rows=1000

/home/r/git_projekty/sparkTraining/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3493: UserWarning: Config option `max_num_rows` not recognized by `SparkSql`.
  if await self.run_code(code, result, async_=asy):


In [2]:
# general paths
DATA = '/home/r/git_projekty/sparkTraining/data'
PACKAGED_DATA = '/home/r/git_projekty/sparkTraining/data/packaged'
UNPACKAGED_DATA = '/home/r/git_projekty/sparkTraining/data/unpackaged'
UNPACKAGED_DATA_FOLDERS = os.listdir(UNPACKAGED_DATA)

In [3]:
print(UNPACKAGED_DATA_FOLDERS)

['transactions-fraud-datasets', 'csecicids2018', 'us-wildfire-records-6th-edition', 'game-recommendations-on-steam']


In [15]:
def check_extension_file(filepath):
    _, file_extension = os.path.splitext(filepath)
    return file_extension.lower()[1:]

def check_file_name(filepath):
    file_name, _ = os.path.splitext(filepath)
    return file_name

In [16]:
for folder in UNPACKAGED_DATA_FOLDERS:
    print(folder)
    folder_path = os.path.join(UNPACKAGED_DATA, folder)
    files = os.listdir(folder_path)
    for file in files:
        file_path = os.path.join(folder_path, file)
        # print(f'file: {file} file_path {file_path}' )
        print(check_extension_file(file), check_file_name(file))

transactions-fraud-datasets
csv cards_data
json train_fraud_labels
json mcc_codes
csv users_data
csv transactions_data
csecicids2018
parquet Botnet-Friday-02-03-2018_TrafficForML_CICFlowMeter
parquet DDoS1-Tuesday-20-02-2018_TrafficForML_CICFlowMeter
parquet Infil2-Thursday-01-03-2018_TrafficForML_CICFlowMeter
parquet DoS2-Friday-16-02-2018_TrafficForML_CICFlowMeter
parquet DoS1-Thursday-15-02-2018_TrafficForML_CICFlowMeter
parquet Infil1-Wednesday-28-02-2018_TrafficForML_CICFlowMeter
parquet Bruteforce-Wednesday-14-02-2018_TrafficForML_CICFlowMeter
parquet DDoS2-Wednesday-21-02-2018_TrafficForML_CICFlowMeter
parquet Web2-Friday-23-02-2018_TrafficForML_CICFlowMeter
parquet Web1-Thursday-22-02-2018_TrafficForML_CICFlowMeter
us-wildfire-records-6th-edition
csv data
csv _variable_descriptions
sqlite data
game-recommendations-on-steam
json games_metadata
csv games
csv recommendations
csv users


# CSV

In [ ]:
# READ
df_csv = spark.read.option("header", "true").option("inferSchema", "true").csv("data.csv")

# WRITE
df_csv.write.mode("overwrite").option("header", "true").csv("output_csv")


# JSON

In [ ]:
# READ
df_json = spark.read.json("data.json")

# WRITE
df_json.write.mode("overwrite").json("output_json")


# PARQUET

In [ ]:
# READ
df_parquet = spark.read.parquet("data.parquet")

# WRITE (z kompresją snappy - domyślna)
df_parquet.write.mode("overwrite").parquet("output_parquet")

# EXCEL

In [ ]:
# SPARK NEEDS LIBRARY (np. com.crealytics:spark-excel).

# READ
df_excel = spark.read.format("com.crealytics.spark.excel") \
    .option("header", "true") \
    .load("data.xlsx")

In [ ]:
# Zapis jako Delta Table
df_parquet.write.format("delta").mode("overwrite").save("/mnt/delta/table")

# Odczyt konkretnej wersji danych (Time Travel)
df_version = spark.read.format("delta").option("versionAsOf", 0).load("/mnt/delta/table")

In [ ]:
# Odczyt
df_avro = spark.read.format("avro").load("data.avro")

In [ ]:
# import psycopg2

try:
    connection = psycopg2.connect(
        user="postgres",
        password="password",
        host="127.0.0.1",
        port="5432",
        database="postgrestest"
    )

    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    record = cursor.fetchone()
    print(f"Połączono z: {record}")

    # 2. Listowanie baz danych
    print("Dostępne bazy danych:")
    cursor.execute("SELECT datname FROM pg_database WHERE datistemplate = false;")
    databases = cursor.fetchall()

    for db in databases:
        print(f" - {db[0]}")

except Exception as error:
    print(f"Błąd połączenia: {error}")

finally:
    if 'connection' in locals() and connection:
        cursor.close()
        connection.close()
        print("Połączenie zamknięte.")

Połączono z: ('PostgreSQL 16.11 (Ubuntu 16.11-0ubuntu0.24.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 13.3.0-6ubuntu2~24.04) 13.3.0, 64-bit',)
Połączenie zamknięte.


In [ ]:
# import psycopg2
# from psycopg2 import sql

class PostgresManager:
    def __init__(self, config):
        """Config powinien być słownikiem z kluczami: user, password, host, port, database"""
        self.config = config

    def execute_query(self, query, params=None, fetch=False):
        """Uniwersalna metoda do SELECT, INSERT, UPDATE, DELETE"""
        result = None
        try:
            with psycopg2.connect(**self.config) as conn:
                with conn.cursor() as cur:
                    cur.execute(query, params)
                    if fetch:
                        result = cur.fetchall()
                    conn.commit()
            return result
        except Exception as e:
            print(f"Błąd bazy danych: {e}")
            return None

# --- UŻYCIE W JUPYTERZE ---

config = {
    "user": "postgres",
    "password": "password",
    "host": "127.0.0.1",
    "port": "5432",
    "database": "postgrestest"
}

db = PostgresManager(config)

# # Przykład 1: Pobieranie danych (DQL)
# dbs = db.execute_query("SELECT datname FROM pg_database", fetch=True)
# print(dbs)

# # Przykład 2: Tworzenie tabeli (DDL)
# db.execute_query("CREATE TABLE IF NOT EXISTS users (id serial PRIMARY KEY, name varchar(100));")

In [ ]:
# Przykład 1: Pobieranie danych (DQL)
# dbs = db.execute_query("SELECT datname FROM pg_database", fetch=True)
# print(dbs)

# # Przykład 2: Tworzenie tabeli (DDL)
# db.execute_query("CREATE TABLE IF NOT EXISTS users (id serial PRIMARY KEY, name varchar(100));")